In [1]:
import pandas as pd 
import json 
import requests
import os 
from dotenv import load_dotenv
import sys 

def print_json(data:json):
    """json 형태의 데이터를 출력하는 함수입니다.

    Args:
        data (json): json 형태의 데이터
    """
    pretty_json = json.dumps(data, indent=4)
    print(pretty_json)

def find_target_directory(target_dir_name):
    current_path = os.getcwd()
    while True:
        # 상위 디렉토리에서 대상 디렉토리 이름을 찾음
        parent_path, current_dir = os.path.split(current_path)
        
        if current_dir == target_dir_name:
            return current_path
        
        if parent_path == current_path:  # 최상위 디렉토리에 도달하면 종료
            break
        
        # 상위 디렉토리로 이동
        current_path = parent_path
    
    return None  # 대상 디렉토리를 찾지 못한 경우


def get_project_root_path(proejct_directory_name: str='odysseyes'):
    return  find_target_directory(target_dir_name=proejct_directory_name)


project_root_path = get_project_root_path()
print(f"프로젝트 경로: {project_root_path}")
sys.path.append(os.path.join(project_root_path, 'recommend', 'func'))

from tmap_route_optimizer import TMAPClient, RouteOptimizer, PlaceDataManager

프로젝트 경로: /Users/jaehwan/SSAFY/odysseyes


In [2]:
load_dotenv()
tmap_api_key = os.getenv('SK_OPEN_API_KEY')
tmap_client = TMAPClient(tmap_api_key)
place_data_manager = PlaceDataManager()
route_optimizer = RouteOptimizer(tmap_client=tmap_client, place_data_manager=place_data_manager)

In [3]:
place_combination_list = place_data_manager.generate_place_combinations(region='부여')

In [4]:
start = '대전 서구 월평동 1216'
festival_place = '백제문화단지'
region = '부여'

In [5]:
for i in range(len(place_combination_list)):
    place_combination_list[i] = [start] + place_combination_list[i] + [festival_place]
    # place_combination_list[i].extend([start, festival_place])
    print(place_combination_list[i])

['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '백제문화단지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '궁남지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '부소산성', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '낙화암', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '정림사지5층석탑', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '백제해장국', '백제문화단지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '백제해장국', '궁남지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '백제해장국', '부소산성', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '백제해장국', '낙화암', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '백제해장국', '정림사지5층석탑', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '메밀꽃필무렵', '백제문화단지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '메밀꽃필무렵', '궁남지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '메밀꽃필무렵', '부소산성', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '메밀꽃필무렵', '낙화암', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '메밀꽃필무렵', '정림사지5층석탑', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '솔내음', '백제문화단지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '솔내음', '궁남지', '백제문화단지']
['대전 서구 월평동 1216', '무드빌리지', '솔내음', '부소산성', '백제문화단지'

In [6]:
print(len(place_combination_list))

125


In [7]:
for place_comb in place_combination_list:
    routes_data = dict()
    print(place_comb)
    for i, start in enumerate(place_comb):
        for j, end in enumerate(place_comb):
            if i != j :
                print(f'{place_comb[i]} -> {place_comb[j]}')
                if i > 0:
                    start_poi = tmap_client.get_poi(keyword=place_comb[i], region=region)
                else:
                    start_poi = tmap_client.get_poi(keyword=place_comb[i])
                if j > 0:
                    end_poi = tmap_client.get_poi(keyword=place_comb[j], region=region)
                else:
                    end_poi = tmap_client.get_poi(keyword=place_comb[j])

                print(start_poi)
                print(end_poi)

                routes_data[(i, j)] = tmap_client.get_route_data(start=start_poi, end=end_poi)
                # print_json(routes_data[(i, j)])
    # for testing
    break

['대전 서구 월평동 1216', '무드빌리지', '장원막국수', '백제문화단지', '백제문화단지']
대전 서구 월평동 1216 -> 무드빌리지
{'latitude': '36.35496178', 'longitude': '127.35677043', 'name': '대전 서구 월평동 1216'}
{'latitude': '36.29285019', 'longitude': '126.92453303', 'name': '무드빌리지'}
대전 서구 월평동 1216 -> 장원막국수
{'latitude': '36.35496178', 'longitude': '127.35677043', 'name': '대전 서구 월평동 1216'}
{'latitude': '36.28912812', 'longitude': '126.90875677', 'name': '장원막국수'}
대전 서구 월평동 1216 -> 백제문화단지
{'latitude': '36.35496178', 'longitude': '127.35677043', 'name': '대전 서구 월평동 1216'}
{'latitude': '36.30662608', 'longitude': '126.90670093', 'name': '백제문화단지'}
대전 서구 월평동 1216 -> 백제문화단지
{'latitude': '36.35496178', 'longitude': '127.35677043', 'name': '대전 서구 월평동 1216'}
{'latitude': '36.30662608', 'longitude': '126.90670093', 'name': '백제문화단지'}
무드빌리지 -> 대전 서구 월평동 1216
{'latitude': '36.29285019', 'longitude': '126.92453303', 'name': '무드빌리지'}
{'latitude': '36.35496178', 'longitude': '127.35677043', 'name': '대전 서구 월평동 1216'}
무드빌리지 -> 장원막국수
{'latitude': '36.29

In [8]:
print(len(routes_data))

20


In [9]:
print_json(routes_data[0,1])

{
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": {
                "type": "Point",
                "coordinates": [
                    127.35686486460436,
                    36.35506787940311
                ]
            },
            "properties": {
                "totalDistance": 69080,
                "totalTime": 3420,
                "totalFare": 2700,
                "taxiFare": 80050,
                "index": 0,
                "pointIndex": 0,
                "name": "",
                "description": "\uc6d4\ud3c9\uc0c8\ub738\ub85c4\ubc88\uae38 \uc744 \ub530\ub77c  \ubc29\uba74\uc73c\ub85c 24m \uc774\ub3d9",
                "nextRoadName": "\uc6d4\ud3c9\uc0c8\ub738\ub85c4\ubc88\uae38",
                "turnType": 200,
                "pointType": "S"
            }
        },
        {
            "type": "Feature",
            "geometry": {
                "type": "LineString",
                "coordina

### routes_data -> 하나의 조합에 대한 1대1 경로 정보 딕셔너리

각각의 1대1 경로 정보에 대해서 '전체 거리' + '전체 시간' + '전체 비용' 합산 점수 산정

In [ ]:
from itertools import permutations
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict

def get_scaled_properties(routes:dict):
    routes = list(routes.values())
    all_features_list = [route['features'][0] for route in routes]  # 모든 1대1 경로들의 features 리스트
    all_properties_list = [features['properties'] for features in all_features_list]  # 모든 1대1 경로들의 경로 요약 정보 

    # 경로들의 전체 거리, 소요 시간, 비용만 추출한 리스트 
    selected_properties_data = [
        {
            'totalDistance': properties['totalDistance'],
            'totalTime': properties['totalTime'],
            'totalFare': properties['totalFare'] 
        }
        for properties in all_properties_list
    ]

    scaler = MinMaxScaler()

    scaled_selected_properties_data = scaler.fit_transform(pd.DataFrame(selected_properties_data))
    scaled_scores = []
    for i in range(len(scaled_selected_properties_data[0])):
        scaled_score = []
        for j in range(len(scaled_selected_properties_data)):
            scaled_score.append(round(float(scaled_selected_properties_data[j][i]), 3))
        scaled_scores.append(scaled_score)
    
    scaledProperties = []
    totalRouteScores = []
    for i in range(len(scaled_scores[0])):
        scaledProperty = {
            'scaledDistance': round(1 - scaled_scores[0][i], 2),
            'scaledTime': round(1 - scaled_scores[1][i], 2),
            'scaledFare': round(1 - scaled_scores[2][i], 2),
            # 'scaledPlaceScore': scaled_scores[3][i],
        }
        
        scaledDTFScore = round(sum(scaledProperty.values()), 2)
        scaledProperty['scaledDTFScore'] = scaledDTFScore
        # totalRouteScores.append(totalRouteScore)
        scaledProperties.append(scaledProperty)

    for i, route in enumerate(routes):
        # 기존 route 정보와 새로운 속성 추가 후 정렬
        # ordered_route = OrderedDict([
        #     ('properties', route.get('properties')),
        #     ('scaledProperties', scaledProperties[i]),
        #     # ('totalRouteScore', totalRouteScores[i]),
        #     ('points', route.get('points')),
        #     ('paths', route.get('paths')),
        #     ('lineCoordinates', route.get('lineCoordinates'))
        # ])
        routes[i]['features'][0]['properties']['scaledProperties'] = scaledProperties[i]
        # routes[i]['features'][0]['properties']['scaledProperties']['scaledDTFScore'] = totalRouteScores[i] 
    
    return routes


# 각 경로에 대한 점수를 계산하는 함수
def calculate_route_score(route, routes_data, weight_distance=0.4, weight_time=0.4, weight_cost=0.2):
    score = 0
    for i in range(len(route) - 1):
        route_info = routes_data[(route[i], route[i + 1])]
        score += (
            weight_distance * route_info["distance"] +
            weight_time * route_info["time"] +
            weight_cost * route_info["cost"]
        )
    return score

# 출발지에서 시작하여 모든 장소를 방문 후 출발지로 돌아오는 최적 경로 탐색
def find_optimal_route(places, routes_data):
    num_places = len(places)
    all_routes = permutations(range(1, num_places))  # 출발지를 제외한 경유지들의 순열 생성
    best_score = float('inf')
    best_route = None
    
    for route in all_routes:
        route = (0,) + route + (0,)  # 출발지(0)를 추가해 순환 경로 형성
        score = calculate_route_score(route, routes_data)
        if score < best_score:
            best_score = score
            best_route = route
            
    return best_route, best_score

In [17]:
new_routes_data = get_scaled_properties(routes_data)

In [20]:
print_json(new_routes_data[0]['features'][0])

{
    "type": "Feature",
    "geometry": {
        "type": "Point",
        "coordinates": [
            127.35686486460436,
            36.35506787940311
        ]
    },
    "properties": {
        "totalDistance": 69080,
        "totalTime": 3420,
        "totalFare": 2700,
        "taxiFare": 80050,
        "index": 0,
        "pointIndex": 0,
        "name": "",
        "description": "\uc6d4\ud3c9\uc0c8\ub738\ub85c4\ubc88\uae38 \uc744 \ub530\ub77c  \ubc29\uba74\uc73c\ub85c 24m \uc774\ub3d9",
        "nextRoadName": "\uc6d4\ud3c9\uc0c8\ub738\ub85c4\ubc88\uae38",
        "turnType": 200,
        "pointType": "S",
        "scaledProperties": {
            "scaledDistance": 0.04,
            "scaledTime": 0.07,
            "scaledFare": 0.16,
            "scaledDTFScore": 0.27
        }
    }
}


In [ ]:

# 최적 경로 탐색 수행
best_route, best_score = find_optimal_route(place_combination_list[0], routes_data)
print("최적 경로:", best_route)
print("최적 점수:", best_score)

In [ ]:
import folium

# 지도 시각화 예시
m = folium.Map(location=[places[0]["lat"], places[0]["lon"]], zoom_start=13)

# 최적 경로 표시
for i in range(len(best_route) - 1):
    start = places[best_route[i]]
    end = places[best_route[i + 1]]
    polyline = routes_data[(best_route[i], best_route[i + 1])]["polyline"]
    folium.PolyLine(locations=polyline, color="blue", weight=2.5, opacity=1).add_to(m)

# 시작 및 끝 지점 마커 추가
folium.Marker([places[0]["lat"], places[0]["lon"]], popup="출발지").add_to(m)

# 지도 출력
m
